In [1]:
import subprocess
from pathlib import Path

from IPython.display import Image, Video
from matplotlib import pyplot as plt

from data_loading import load_annotated_6k_collection

In [2]:
tfs = load_annotated_6k_collection()

100%|██████████| 2748/2748 [00:11<00:00, 230.18it/s]


ValueError: Your version of xlrd is 2.0.1. In xlrd >= 2.0, only the xls format is supported. Install openpyxl instead.

In [3]:
import tempfile


def compare_isoform_structures(gene,
                               fname,
                               n_pixel_height=1000,
                               n_frames=100,
                               frame_rate=10,
                               verbose=False,
                               pymol_script_dir='/tmp/pymol_scripts',
                               pymol_session_dir='/tmp/pymol_sessions',
                               structure_dir='../../data/internal/alphafold/casp14',
                               structure_dir_backup='../../data/internal/alphafold/reduced_dbs'):
    """

    Args:
        gene ([type]): [description]
        fname ([type]): [description]
        n_pixel_height (int, optional): [description]. Defaults to 1000.
        n_frames (int, optional): [description]. Defaults to 100.
        frame_rate (int, optional): [description]. Defaults to 10.
        verbose (bool, optional): [description]. Defaults to False.
        pymol_script_dir (str, optional): [description]. Defaults to '/tmp/pymol_scripts'.
        pymol_session_dir (str, optional): [description]. Defaults to '/tmp/pymol_sessions'.

    Raises:
        ValueError: [description]
    """
    if Path(fname).exists():
        print('already have video for: ', gene.name)
        return
    if not fname.endswith('.mp4'):
        raise ValueError('Must be .mp4 output file')
    structure_dir = Path(structure_dir)
    structure_dir_backup = Path(structure_dir_backup)
    pymol_script_dir = Path(pymol_script_dir)
    pymol_session_dir = Path(pymol_session_dir)
    pymol_script_dir.mkdir(exist_ok=True)
    pymol_session_dir.mkdir(exist_ok=True)
    pymol_script_path = pymol_script_dir / '{}_alphafold_cloned_isoforms.pml'.format(gene.name)
    pymol_session_path = pymol_session_dir / '{}_alphafold_cloned_isoforms.pse'.format(gene.name)
    temp_dir = tempfile.TemporaryDirectory()
    isoforms_with_structure = []
    with open(pymol_script_path, 'w') as f:
        f.write('reinitialize\n')
        f.write('bg_color white\n')
        for isoform in gene.cloned_isoforms:
            fpath = structure_dir / (isoform.name + '/ranked_0.pdb')
            fpath_backup = structure_dir_backup / (isoform.name + '/ranked_0.pdb')
            if not fpath.exists():
                if fpath_backup.exists():
                    fpath = fpath_backup
                else:
                    print('Not all structures for ', gene.name)
                    return
            isoforms_with_structure.append(isoform)
            f.write('load {}, {}\n'.format(fpath, isoform.name.replace('-', '_')))
            if isoform.name != gene.cloned_isoforms[0].name:
                f.write('align {}, {}\n'.format(isoform.name.replace('-', '_'),
                                                gene.cloned_isoforms[0].name.replace('-', '_')))
            exon_colors = gene._get_exon_colors()
            exon_pos = 1
            nt_diff = 0
            for exon in isoform.exons:
                n_nt_to_exon = (exon.end - exon.start) + nt_diff
                n_aa_exon = round(n_nt_to_exon / 3)
                nt_diff = {0: 0, 1: 1, 2: -1}[n_nt_to_exon % 3]
                color = exon_colors[(isoform.name, exon.start, exon.end)]
                start = exon_pos
                end = (exon_pos + n_aa_exon) - 1
                color_name = isoform.name + '_' + str(exon.start) + '_color'
                f.write('set_color {}, {}\n'.format(color_name, color))
                f.write('color {}, {} AND resi {}-{}\n'.format(color_name,
                                                            isoform.name.replace('-', '_'),
                                                            start,
                                                            end))
                exon_pos += n_aa_exon

        f.write('set grid_mode, 1\n')
        for i, isoform in enumerate(isoforms_with_structure):
            f.write('set grid_slot, {}, {}\n'.format(i + 1, isoform.name.replace('-', '_')))

        f.write('refresh\n')  # need refresh otherwise grid mode doesn't work
        f.write('viewport {}, {}\n'.format(n_pixel_height * len(isoforms_with_structure), n_pixel_height))
        f.write('zoom\n')
        f.write('save {}\n'.format(pymol_session_path))
        f.write('set cache_frames, 0\n')
        f.write('set ray_trace_frames, 1\n')
        f.write('mset 1-{}\n'.format(n_frames))
        f.write("movie.roll first=1, last={}, axis=y\n".format(n_frames // 2))
        f.write("movie.roll first={}, last=-1, axis=x\n".format(n_frames // 2 + 1))
        f.write('mplay\n')
        f.write('mpng {}/movie\n'.format(temp_dir.name))
    print('Making video for ', gene.name)
    ret = subprocess.run('pymol -c {}'.format(pymol_script_path),
                         capture_output=True,
                         shell=True)
    if verbose:
        print(ret.stdout.decode())
        print(ret.stderr.decode())
    Path(fname).parents[0].mkdir(exist_ok=True, parents=True)
    # crf is between 0 and 51 and controls compression with 0 being lossless
    # I tested and found 27 was good, 30 looked bad
    cmd = 'ffmpeg -y -r {} -f image2 -i {}/movie%04d.png -c:v libx264 -profile:v main -crf 27 -tune stillimage -pix_fmt yuv420p {}'.format(frame_rate, temp_dir.name, fname)
    ret = subprocess.run(cmd, capture_output=True, shell=True)
    if verbose:
        print(cmd)
        print(ret.stdout.decode())
        print(ret.stderr.decode())
    temp_dir.cleanup()


In [5]:
import tqdm
fpath_template = '../../figures/alphafold/videos/{}_alphafold2_cloned-isoforms.mp4'
for tf in tqdm.tqdm(tfs.values()):
        compare_isoform_structures(gene=tf,
                                   fname=fpath_template.format(tf.name),
                                   n_frames=60,
                                   frame_rate=6)

  0%|          | 0/309 [00:00<?, ?it/s]

Not all structures for  ZBTB18
already have video for:  PRRX1
Making video for  POU2F1


  1%|          | 3/309 [03:20<5:40:25, 66.75s/it]

Making video for  ATF3


  1%|▏         | 4/309 [04:19<5:27:33, 64.44s/it]

...

Making video for  NKRF


 99%|█████████▉| 306/309 [5:19:18<02:43, 54.36s/it]

Making video for  HDX


100%|██████████| 309/309 [5:20:36<00:00, 62.25s/it]

Not all structures for  ZFY
already have video for:  HSFY1
